In [23]:
# import dependencies
import pandas as pd
import numpy as np
import gmaps
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Turn off chained warning
pd.options.mode.chained_assignment = None

# import key
from config.config import google_api_key

# configure gmaps
gmaps.configure(api_key=google_api_key)

In [24]:
# load data
hpi2_df = pd.read_csv('../data/HPI2_MasterFile_2021-04-22.csv')
ca_census_tract_df = pd.read_csv('../resource/2020_gaz_tracts_06.txt', sep='\t', lineterminator='\n')

# merge geographical information into dataframe
hpi2_df = hpi2_df.merge(ca_census_tract_df, left_on='CensusTract', right_on='GEOID')
hpi2_df

,CensusTract,pop2010,pct2010gq,City,ZIP,County_FIPS,County_me,UrbanType,hpi2score,hpi2_pctile_pos,...,other_pct,version,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG\r
0,6001400100,2937,0.1,Oakland,94704.0,6001,Alameda,urban_area,1.182028,99.114590,...,0.851209,4/22/2021,CA,6001400100,6945857,0,2.682,0.000,37.867656,-122.231881
1,6001400200,1974,3.5,Oakland,94618.0,6001,Alameda,urban_area,1.325874,99.704863,...,0.151976,4/22/2021,CA,6001400200,586560,0,0.226,0.000,37.848138,-122.249592
2,6001400300,4865,0.8,Oakland,94618.0,6001,Alameda,urban_area,0.925666,95.932247,...,0.411100,4/22/2021,CA,6001400300,1110840,0,0.429,0.000,37.840584,-122.254478
3,6001400400,3703,0.9,Oakland,94609.0,6001,Alameda,urban_area,1.117603,98.537149,...,0.567108,4/22/2021,CA,6001400400,716138,0,0.277,0.000,37.848284,-122.257445
4,6001400500,3517,3.6,Oakland,94609.0,6001,Alameda,urban_area,0.562799,83.318363,...,0.653966,4/22/2021,CA,6001400500,591424,0,0.228,0.000,37.848545,-122.264735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6875,6115040500,4111,0.8,Olivehurst,95961.0,6115,Yuba,urban_area,-0.857437,5.671757,...,0.072975,4/22/2021,CA,6115040500,2438067,0,0.941,0.000,39.100516,-121.553801
6876,6115040600,6130,0.1,Olivehurst,95961.0,6115,Yuba,urban_area,-0.673751,12.293084,...,0.179445,4/22/2021,CA,6115040600,4164409,0,1.608,0.000,39.084246,-121.546824
6877,6115040800,4233,0.1,NaN,95692.0,6115,Yuba,urban_cluster,-0.037308,47.427178,...,0.188991,4/22/2021,CA,6115040800,109394813,128316,42.238,0.050,39.041053,-121.433980
6878,6115040901,2783,0.4,NaN,95901.0,6115,Yuba,rural,-0.425301,24.188374,...,0.000000,4/22/2021,CA,6115040901,263666616,4473094,101.802,1.727,39.185210,-121.358537


In [25]:
# remove null value rows
hpi2_df = hpi2_df.loc[hpi2_df['hpi2score'].isnull().values == False]

# rename pa to Napa county
hpi2_df.loc[hpi2_df['County_me'] == 'pa', 'County_me'] = 'Napa'

In [26]:
county_list = hpi2_df['County_me'].unique()

outlier_df = pd.DataFrame(columns=['County', 'q1', 'q3', 'iqr'])

# calculate iqr, q1, and q3 for each county
for i, county in enumerate(county_list):
    county_df = hpi2_df.loc[hpi2_df['County_me'] == county]
    q1 = county_df['insured'].quantile(0.25)
    q3 = county_df['insured'].quantile(0.75)

    iqr = q3 - q1
    outlier_df.loc[i] = [county, q1, q3, iqr]


In [27]:
# merge outlier dataset into dataframe
hpi2_df = hpi2_df.merge(outlier_df, left_on='County_me', right_on='County')

In [28]:
# Calculate if field is an outlier
hpi2_df['is_outlier'] = np.where((hpi2_df['insured'] < hpi2_df['q1'] - 1.5 * hpi2_df['iqr']) |
                                 (hpi2_df['insured'] > hpi2_df['q3'] + 1.5 * hpi2_df['iqr']), True, False)

# Remove outlier fields
hpi2_df = hpi2_df.loc[hpi2_df['is_outlier'] == False]

In [30]:
# set latitude and longitude for heatmap
coordinates = hpi2_df[['INTPTLAT', 'INTPTLONG\r']]
# set the weights to the percentile of people insured
# subtracted from 100 to flip the coloring scheme where worst peforming areas in red
insured_pctile = 100-hpi2_df['insured_pctile']

**Heatmap of Percentile of Insured Population between Ages 18-64 by Census Tract**

In [31]:
# map heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights = insured_pctile, dissipating=True, max_intensity=100)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
insured_df = hpi2_df.groupby('County_me').agg({
                                               'insured': [
                                                   np.mean,
                                                   np.median,
                                                   'min',
                                                   'max'
                                               ]
                                            })

insured_df.rename_axis('County Name', inplace=True)

insured_df.columns = ['_'.join(index_pair) for index_pair in insured_df.columns]

# rename columns
insured_df.rename(columns={
    'insured_mean': 'Average Insured %',
    'insured_median': 'Median Insured %',
    'insured_min': 'Minimun Insured %',
    'insured_max': 'Maximum Insured %'
}, inplace = True)

# insured_df.sort_values(('Insured', 'median'))
insured_df


,Average Insured %,Median Insured %,Minimun Insured %,Maximum Insured %
County Name,,,,
Alameda,86.648036,88.10,63.5,99.6
Amador,87.800000,87.85,85.5,90.5
Butte,82.462500,82.90,71.2,95.3
Calaveras,81.675000,81.45,80.3,83.5
Colusa,74.075000,74.00,71.5,76.8
Contra Costa,86.778824,88.75,60.2,99.5
Del Norte,79.580000,81.40,68.1,87.2
El Dorado,89.293333,91.10,73.0,97.2
Fresno,75.465031,76.10,48.4,95.5


In [33]:
# Load county coordinates from file
county_coordinates_df = pd.read_csv('../resource/CenPop2010_Mean_CO06.txt', )

In [34]:
# Combine data insured dataset together with lat and lng of county center
insured_df = insured_df.merge(county_coordinates_df, left_index=True, right_on='COUNAME')

In [35]:
coordinates = insured_df[['LATITUDE', 'LONGITUDE']]
insured_median = 100-insured_df['Median Insured %']

**Heatmap of the Median % Insured between Ages 18-64 By County**

In [36]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=insured_median, dissipating=False, point_radius=.6, max_intensity=30)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))